In [1]:
import cv2
import cvzone
from cvzone.HandTrackingModule import HandDetector

ModuleNotFoundError: No module named 'cv2'

In [29]:
def all_fingers_raised(lmList):

    if len(lmList) != 21:
        return False
        
    thumb_tip = 4
    thumb_ip = 3
    index_tip = 8
    index_ip = 7
    middle_tip = 12
    middle_ip = 11
    ring_tip = 16
    ring_ip = 15
    pinky_tip = 20
    pinky_ip = 19
    return (is_finger_raised(lmList, thumb_tip, thumb_ip) and
            is_finger_raised(lmList, index_tip, index_ip) and
            is_finger_raised(lmList, middle_tip, middle_ip) and
            is_finger_raised(lmList, ring_tip, ring_ip) and
            is_finger_raised(lmList, pinky_tip, pinky_ip))

def is_finger_raised(lmList, tip, phalanx):
    return lmList[tip][1] < lmList[phalanx][1]

In [39]:
hand_detector = HandDetector(detectionCon=0)
cap = cv2.VideoCapture(0)
cap.set(3, 900)
cap.set(4, 700)

center_x, center_y, width, height = 100, 100, 300, 300


while True:
    rectangle_color = (255, 0, 255)
    ret, frame = cap.read()
    frame = cv2.flip(frame, 1)
    frame = hand_detector.findHands(frame)
    lmList, _ = hand_detector.findPosition(frame)

    if lmList:
        # x and y of tip of your finger
        index_middle_distance, _, _ = hand_detector.findDistance(8, 12, frame)

        if all_fingers_raised(lmList):
            rectangle_color = (0, 0, 255)
        
        if index_middle_distance < 40:
            cursor = lmList[8]
            if center_x - width //2 < cursor[0] < center_x+width //2 and center_y - height //2 < cursor[1] < center_y+height //2:
                rectangle_color = (255, 255, 255)
                center_x, center_y = cursor

    cv2.rectangle(frame, (center_x - width//2, center_y - height//2),
                  (center_x+width//2,center_y+height//2), rectangle_color, cv2.FILLED)
    cv2.imshow("cam", frame)
    
    wait_key = cv2.waitKey(1)
    if wait_key == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()

I0000 00:00:1725140152.896456    8123 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1725140152.897560   10895 gl_context.cc:357] GL version: 3.2 (OpenGL ES 3.2 Mesa 24.0.3-1pop1~1711635559~22.04~7a9f319), renderer: Mesa Intel(R) UHD Graphics 620 (WHL GT2)


In [14]:
import pyautogui, tkinter
import cv2
from cvzone.HandTrackingModule import HandDetector  # Assuming you're using this for hand tracking

hand_detector = HandDetector(detectionCon=0)
cap = cv2.VideoCapture(0)
cap.set(3, 900)
cap.set(4, 700)

center_x, center_y, width, height = 100, 100, 300, 300
dragging = False

while True:
    rectangle_color = (255, 0, 255)
    ret, frame = cap.read()
    frame = cv2.flip(frame, 1)
    frame = hand_detector.findHands(frame)
    lmList, _ = hand_detector.findPosition(frame)

    if lmList:
        # x and y of tip of your finger
        index_middle_distance, _, _ = hand_detector.findDistance(8, 12, frame)

        # Check if all fingers are raised
        if all_fingers_raised(lmList):
            rectangle_color = (0, 0, 255)
            dragging = False  # Reset dragging when hand is fully open
        
        # Pinch gesture to "grab" object
        if index_middle_distance < 40:
            cursor = lmList[8]
            if center_x - width // 2 < cursor[0] < center_x + width // 2 and center_y - height // 2 < cursor[1] < center_y + height // 2:
                rectangle_color = (255, 255, 255)
                center_x, center_y = cursor  # Move the box
                dragging = True
                
                # Simulate mouse movement
                screen_x, screen_y = pyautogui.size()  # Get screen dimensions
                screen_cursor_x = int(cursor[0] / 900 * screen_x)
                screen_cursor_y = int(cursor[1] / 700 * screen_y)
                pyautogui.moveTo(screen_cursor_x, screen_cursor_y)

        # Drop object when gesture is released
        if not dragging:
            pyautogui.mouseUp()

    cv2.rectangle(frame, (center_x - width // 2, center_y - height // 2),
                  (center_x + width // 2, center_y + height // 2), rectangle_color, cv2.FILLED)
    cv2.imshow("cam", frame)
    
    wait_key = cv2.waitKey(1)
    if wait_key == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()


AttributeError: 'tuple' object has no attribute 'tb_frame'